# Text-to-SQL solution on top of Amazon products

In this notebook we will explore what it takes to implement a text-to-SQL solution on top of our Amazon dataset using LlamaIndex.

## Imports

In [1]:
import openai
import pandas as pd

from superlinked_app.config import settings

settings.validate_processed_dataset_exists()

2024-12-12 18:33:52.629 | INFO     | superlinked_app.config:<module>:9 - Loading '.env' file from: /Users/pauliusztin/Documents/01_projects/hands-on-retrieval/.env


In [2]:
openai.api_key = settings.OPENAI_API_KEY

## Create Database Schema

In [3]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    select,
)

In [4]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

Create the product SQL table:

In [5]:
product_table_name = "product"
product_table = Table(
    product_table_name,
    metadata_obj,
    Column("asin", String(255), primary_key=True),
    Column("type", String(255)),
    Column("title", String(255)),
    Column("description", String(1000)),
    Column("review_rating", Float),
    Column("review_count", Integer),
    Column("price", Float),
)
metadata_obj.create_all(engine)

## Define SQL Database

We first define our SQLDatabase abstraction (a light wrapper around SQLAlchemy).

In [6]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [7]:
llm = OpenAI(temperature=0.1, model=settings.OPENAI_MODEL_ID)

In [8]:
sql_database = SQLDatabase(engine, include_tables=[product_table_name])

Add the data to the SQL database:

In [9]:
df = pd.read_json(settings.PROCESSED_DATASET_PATH, lines=True).drop(columns=['category'])
df.head()

,asin,type,title,description,price,review_rating,review_count
0,B07WP4RXHY,product,YUEPIN U-Tube Clamp 304 Stainless Steel Hose P...,Product Description Specification: Material: 3...,9.99,4.7,54
1,B07VRZTK2N,product,"Apron for Women, Waterproof Adjustable Bib Coo...",,11.99,4.0,152
2,B07V2F5SN1,product,DIY 5D Diamond Painting by Number Kit for Adul...,Product Description 5D DIY Diamond Painting is...,9.99,4.6,378
3,B08GM4LMKN,product,Órgano bucal de armónica trémolo de 24 agujero...,Descripción del producto Especificación: Tipo ...,25.63,4.6,9
4,B00MNLQQ7K,product,"Design Toscano QM2787100 Darby, the Forest Faw...",,40.72,4.7,274


In [10]:
for _, row in df.iterrows():
    stmt = insert(product_table).values(**row.to_dict())
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

## Query Database

Let's query the database to see that everything worked fine.

In [11]:
stmt = select(
    product_table.c.asin,
    product_table.c.type,
    product_table.c.title,
    product_table.c.description,
    product_table.c.review_rating,
    product_table.c.review_count,
    product_table.c.price,
).select_from(product_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchmany(3)
    for result in results:
        print(f"ASIN: {result.asin}")
        print(f"Type: {result.type}")
        print(f"Title: {result.title}")
        print(f"Description: {result.description[:100]}...")  # Truncate long descriptions
        print(f"Rating: {result.review_rating:.1f}/5.0 ({result.review_count} reviews)")
        print(f"Price: ${result.price:.2f}")
        print("-" * 80)

ASIN: B07WP4RXHY
Type: product
Title: YUEPIN U-Tube Clamp 304 Stainless Steel Hose Pipe Cable Strap Clips With Rubber Cushioned (1-21/32"(42mm)-10pcs)
Description: Product Description Specification: Material: 304 Stainless Steel,100% New Rubber Color: Silver Shape...
Rating: 4.7/5.0 (54 reviews)
Price: $9.99
--------------------------------------------------------------------------------
ASIN: B07VRZTK2N
Type: product
Title: Apron for Women, Waterproof Adjustable Bib Cooking Aprons with Pocket-2 Side Coral Velvet Towels Stitched Durable Pinstripe Aprons for Dishwashing, Baking, Grill, Restaurant Even Garden Craft
Description: ...
Rating: 4.0/5.0 (152 reviews)
Price: $11.99
--------------------------------------------------------------------------------
ASIN: B07V2F5SN1
Type: product
Title: DIY 5D Diamond Painting by Number Kit for Adult, Full Drill Crystal Rhinestone Embroidery Cross Stitch Diamond Embroidery Dotz Kit Home Wall Decor 15.8×11.8 Inch (Wolf)
Description: Product Descripti

## Text-to-SQL Retriever

Let's use the LlamaIndex SQL retriever to map text to SQL instructions and retrieve data from the SQLite table. 

In [12]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.response.notebook_utils import display_source_node

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=[product_table_name], return_raw=False
)

All the content is in the `metadata`:

In [13]:
results = nl_sql_retriever.retrieve(
   "books with a price lower than 100"
)
for n in results[:3]:
    display_source_node(n, show_source_metadata=True)

**Node ID:** fe83bde8-54c6-46a7-bd31-7661bab4f8a8<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'All Aboard! New York: A City Primer', 'price': 9.99}<br>

**Node ID:** 5ab49fa1-e254-4031-93bd-30dd0921c4ae<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Feminist Baby (Feminist Baby, 4)', 'price': 10.59}<br>

**Node ID:** cc2142f6-58bf-44ee-9da5-e1df181c0d13<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'The Mindful Dragon: A Dragon Book about Mindfulness. Teach Your Dragon To Be Mindful. A Cute Children Story to Teach Kids about Mindfulness, Focus and Peace. (My Dragon Books)', 'price': 11.69}<br>

In [14]:
results = nl_sql_retriever.retrieve(
   "books with a price lower than 100 and a rating bigger than 4"
)
for n in results[:3]:
    display_source_node(n, show_source_metadata=True)

**Node ID:** bbb01131-a9ca-4b8f-a06a-e5b713bbe0ff<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Stables: Beautiful Paddocks, Horse Barns, and Tack Rooms', 'price': 53.1, 'review_rating': 4.7}<br>

**Node ID:** a0c8a2f9-595b-4fb4-b140-0a0bf68f436f<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'The Mindful Dragon: A Dragon Book about Mindfulness. Teach Your Dragon To Be Mindful. A Cute Children Story to Teach Kids about Mindfulness, Focus and Peace. (My Dragon Books)', 'price': 11.69, 'review_rating': 4.7}<br>

**Node ID:** c1b7b9c7-1ceb-43ee-b7b9-b93990352772<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Build Your Running Body (A Total-Body Fitness Plan for All Distance Runners, from Milers to Ultramarathoners—Run Farther, Faster, and Injury-Free)', 'price': 13.49, 'review_rating': 4.7}<br>

In [15]:
results = nl_sql_retriever.retrieve(
   "Return the top 5 books (along with their review count and price) with the highest reviews rating."
)
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 14d164c1-9926-486e-8616-4e945a345174<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Stables: Beautiful Paddocks, Horse Barns, and Tack Rooms', 'review_count': 100, 'price': 53.1}<br>

**Node ID:** b9a67ec0-7101-446d-b36c-bb8435442123<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'The Mindful Dragon: A Dragon Book about Mindfulness. Teach Your Dragon To Be Mindful. A Cute Children Story to Teach Kids about Mindfulness, Focus and Peace. (My Dragon Books)', 'review_count': 623, 'price': 11.69}<br>

**Node ID:** 094e44be-bf37-4d56-9d9d-3045ffbc6004<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'Build Your Running Body (A Total-Body Fitness Plan for All Distance Runners, from Milers to Ultramarathoners—Run Farther, Faster, and Injury-Free)', 'review_count': 573, 'price': 13.49}<br>

**Node ID:** 2f8c362a-f7ea-4643-8986-107d0ba7ea5b<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'All Aboard! New York: A City Primer', 'review_count': 74, 'price': 9.99}<br>

**Node ID:** a3c0a9e4-c3b5-482e-853f-d66ac38f7787<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'title': 'The Rise: Creativity, the Gift of Failure, and the Search for Mastery', 'review_count': 204, 'price': 13.79}<br>

## Plug in the RetrieverQueryEngine

We compose our SQL Retriever with our standard RetrieverQueryEngine to synthesize a response. The result is similar to LlamaIndex's `NLSQLTableQueryEngine` class, which does both retrieval and synthetization of the response.

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [17]:
response = query_engine.query(
    "Return the top 5 books (along with their review count and price) with the highest reviews rating."
)

In [18]:
print(str(response))

The Mindful Dragon: A Dragon Book about Mindfulness. Teach Your Dragon To Be Mindful. A Cute Children Story to Teach Kids about Mindfulness, Focus and Peace. (My Dragon Books) - 623 reviews, $11.69
Build Your Running Body (A Total-Body Fitness Plan for All Distance Runners, from Milers to Ultramarathoners—Run Farther, Faster, and Injury-Free) - 573 reviews, $13.49
The Rise: Creativity, the Gift of Failure, and the Search for Mastery - 204 reviews, $13.79
Stables: Beautiful Paddocks, Horse Barns, and Tack Rooms - 100 reviews, $53.1
All Aboard! New York: A City Primer - 74 reviews, $9.99
